<a href="https://colab.research.google.com/github/xxp-nlp/add_tag_algorithm/blob/main/2021_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [312]:
filename = '/content/CiWoTestdata-Correct.txt'

dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):
        cols = line.split(' ')
        self.surface = cols[0] # 表層形(surface)


class Chunk_correct:
    def __init__(self, morphs, dst, id):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.id = id
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.replace('、','')

sentences = []
chunks = []
morphs = []
ids = []
dic_correct = {}
i = 0
with open(filename, mode='r') as f:
  for line in f:  # 1行ずつ読込
    dependancies = dependancy.match(line)
    fields = line.split(' ')


    if fields[0] == '#':
      id = fields[1]
      continue

    # EOSまたは係り受け解析結果でない場合
    if not (line == 'EOS\n' or dependancies):
      morphs.append(Morph(line))


    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif len(morphs) > 0:
      chunks.append(Chunk_correct(morphs, dst, id))
      morphs = []  

    # 係り受け結果の場合
    if dependancies:
      dst = int(dependancies.group(1))

    # EOSで係り受け結果がある場合
    if line == 'EOS\n' and len(chunks) > 0:
      sentences.append(chunks)
      chunks = []
      dic_correct[i] = [sentences,id]
      i+=1

for i, chunk in enumerate(sentences[0]):
    print('{}: {}  係り先:{}'.format(i, chunk.trim_phrase, chunk.dst))

0: 同社では  係り先:5
1: すでに  係り先:5
2: 準役員クラス以上に  係り先:5
3: 能力重視型の  係り先:4
4: 年俸制を  係り先:5
5: 導入している。  係り先:-1


In [313]:
filename = '/content/CiWoTestdata.txt'

dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):
        cols = line.split(' ')
        self.surface = cols[0] # 表層形(surface)


class Chunk_wrong:
    def __init__(self, morphs, dst, id):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.id = id
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.replace('、','')
        self.tag = 'KEEP'




dic_wrong = {}
sentences_wrong = []
chunks = []
morphs = []
ids = []
i = 0
with open(filename, mode='r') as f:
  for line in f:  # 1行ずつ読込
    dependancies = dependancy.match(line)
    fields = line.split(' ')


    if fields[0] == '#':
      id = fields[1]
      continue

    # EOSまたは係り受け解析結果でない場合
    if not (line == 'EOS\n' or dependancies):
      morphs.append(Morph(line))


    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif len(morphs) > 0:
      chunks.append(Chunk_wrong(morphs, dst, id))
      morphs = []  

    # 係り受け結果の場合
    if dependancies:
      dst = int(dependancies.group(1))

    # EOSで係り受け結果がある場合
    if line == 'EOS\n' and len(chunks) > 0:
      sentences_wrong.append(chunks)
      chunks = []
      dic_wrong[i] = [sentences_wrong,id]
      i+=1


for i, chunk in enumerate(sentences_wrong[0]):
    print('{}: {}   {}'.format(i, chunk.trim_phrase, chunk.tag))

0: 同社では   KEEP
1: 準役員クラス以上に   KEEP
2: 能力重視型の   KEEP
3: 年俸制を   KEEP
4: すでに   KEEP
5: 導入している。   KEEP


In [315]:
lenth = min(len(sentences),len(sentences_wrong))


# print(lenth)

correct = []
wrong = []

for i in range(len(sentences)):
  for j in range(len(sentences_wrong)):
    if dic_correct[i][1] == dic_wrong[j][1]:  
      correct.append(sentences[i])
      wrong.append(sentences_wrong[j])
      


In [318]:
# タグ付きアルゴリズム



dic2 = {}


for i in range(lenth):
#  print("sentence",i+1)

  for j, chunk in enumerate(wrong[i]):
    if wrong[i][j].trim_phrase not in dic2:
      dic2[wrong[i][j].trim_phrase] = [] 
    dic2[wrong[i][j].trim_phrase].append(j)
  

  for j, chunk in enumerate(correct[i]):
    if j == len(correct[i])-1:
      break

    for m in range(len(dic2[correct[i][j].trim_phrase])):
      for n in range(len(dic2[correct[i][j+1].trim_phrase])):

        if correct[i][j].dst == j+1 and dic2[correct[i][j].trim_phrase][m] == dic2[correct[i][j+1].trim_phrase][n] - 1: 
          wrong[i][dic2[correct[i][j].trim_phrase][m]].tag = "REDUCE"


        if dic2[correct[i][j].trim_phrase][m] < len(wrong[i])-1 and dic2[correct[i][j].trim_phrase][m] == dic2[correct[i][j+1].trim_phrase][n] + 1: 
          wrong[i][dic2[sentences[i][j].trim_phrase][m]].tag = "SWAP_B"
          wrong[i][dic2[sentences[i][j+1].trim_phrase][n]].tag = "SWAP_F"

  dic2 = {}

#    print(j,sentences[i][j].trim_phrase,sentences[i][j].dst)
#  print("------------------------")

In [317]:
for i in range(lenth):
  print("-------  sentence ", i+1, "-------")
  for j, chunk in enumerate(wrong[i]):
    print('{}: {}: {}'.format(j, wrong[i][j].trim_phrase, wrong[i][j].tag))

流式输出内容被截断，只能显示最后 5000 行内容。
2: 国連安保理常任理事国入りに: REDUCE
3: ついては: KEEP
4: ある。: KEEP
-------  sentence  21 -------
0: 戦争を: KEEP
1: 戦後: REDUCE
2: 五十年と: REDUCE
3: いう: REDUCE
4: ことでは: KEEP
5: 総括すべきだと: REDUCE
6: 思う。: KEEP
-------  sentence  22 -------
0: 不毛な: KEEP
1: 戦争は: KEEP
2: いかに: KEEP
3: 犠牲を: KEEP
4: 多く: REDUCE
5: 出す: KEEP
6: ものであるかを: KEEP
7: 後世に: REDUCE
8: 対して: KEEP
9: 同時に: KEEP
10: 伝えなければならない。: KEEP
-------  sentence  23 -------
0: いろいろな: REDUCE
1: 発言が: KEEP
2: 国際社会: KEEP
3: 特に: REDUCE
4: 東南アジアに: REDUCE
5: 対して: KEEP
6: それを: REDUCE
7: やって: KEEP
8: 初めて: KEEP
9: 日本は: KEEP
10: できるようになる。: KEEP
-------  sentence  24 -------
0: しかし: KEEP
1: 国際社会で: KEEP
2: 名誉: REDUCE
3: ある: REDUCE
4: 地位を: KEEP
5: 前文は: KEEP
6: うたっている。: KEEP
-------  sentence  25 -------
0: 私は: REDUCE
1: 軍拡論者ではないが: KEEP
2: シビリアンコントロールが: SWAP_F
3: 議論しなければ: SWAP_B
4: できない。: KEEP
-------  sentence  26 -------
0: いよいよ: SWAP_F
1: これから: SWAP_B
2: 使えると: REDUCE
3: いう: REDUCE
4: 部分が: KEEP
5: いっぱい: KEEP
6: 不備や: KEEP
7: 時代に: REDUCE
8: 合わない: R